In [ ]:
%pip install -U -q seaborn

In [ ]:
import os
import sys
import torch
from pathlib import Path
import pandas as pd
import numpy as np
# import matplotlib.pyplot as plt
# import seaborn as sns
from typing import Optional, Dict

In [ ]:
pd.set_option("display.max_rows", None)
pd.set_option("display.max_columns", None)
pd.set_option("display.max_colwidth", None)

In [ ]:
def _load_pt_file(pt_path: Path | str) -> torch.Tensor:
    obj = torch.load(pt_path, map_location="cpu")
    if isinstance(obj, torch.Tensor):
        T = obj
    else:
        raise ValueError(f"Unsupported saved object in {pt_path}")

    if T.dim() != 2:
        raise ValueError(f"Expected 2D tensor [L, D], got shape {T.shape} in {p}")

    return T.contiguous()

In [ ]:
_load_pt_file(Path("../smoketest/artifacts/pts/A0A077B1I6_CHIKV.pt")).shape

In [ ]:
T1 = _load_pt_file("../smoketest/artifacts/pts/A8D0M1_ADE02.pt")
T1

In [ ]:
T1.shape

In [ ]:
def _seq_stats(T: torch.Tensor) -> Dict[str, float | int]:
    L, D = T.shape
    norms = torch.linalg.vector_norm(T, ord=2, dim=1)
    return {"L": int(L), "D": int(D), 
            "norm_min": float(norms.min()), 
            "norm_max": float(norms.max()), 
            "norms_mean": float(norms.mean()), 
            "norms_std": float(norms.std(unbiased=False))}

In [ ]:
T1_stats = _seq_stats(T1)
T1_stats

In [ ]:
def cmd_summary(per_seq_dir: Path | str, out_csv: Path | str) -> None:
    rows = []
    for pt in sorted(per_seq_dir.glob("*.pt")):
        try:
            T = _load_pt_file(pt)
            s = _seq_stats(T)
            s["file"] = str(pt)
            s["id"] = pt.stem
            rows.append(s)
        except Exception as e:
            rows.append({"file": str(pt), "id": pt.stem, "error": str(e)})

    df = pd.DataFrame(rows)
    df.to_csv(out_csv, index=False)
    print(f"Wrote summary to {out_csv} with {len(df)} rows")
    print(df.head().to_string(index=False))

In [ ]:
cmd_summary(Path("../smoketest/artifacts/pts"), Path("../smoketest/artifacts/analysis.csv"))

In [ ]:
def cmd_residue_norms(pt_file: Path | str, export_csv: Optional[Path | str] = None) -> None:
    T = _load_pt_file(pt_file)
    norms = torch.linalg.vector_norm(T, ord=2, dim=1).numpy()
    df = pd.DataFrame({"i": np.arange(len(norms), dtype=int), "l2_norm": norms})
    if export_csv:
        df.to_csv(export_csv, index=False)
        print(f"Wrote residue norms to {export_csv}  |  L={len(norms)}")
    else:
        print(df.head(10).to_string(index=False))
        print(f"   L={len(norms)}  |  mean={norms.mean():.3f}  |  std={norms.std():.3f}")

In [ ]:
per_seq_dir = Path("../smoketest/artifacts/pts")
for pt in sorted(per_seq_dir.glob("*.pt")):
    cmd_residue_norms(pt)#, export_csv=Path(f"./smoketest/artifacts/pts/norms/{pt.stem}.csv"))

In [ ]:
# cosine similarity
def _cos(a: torch.Tensor, b: torch.Tensor) -> float:
    a, b = a.float(), b.float()
    a_n = torch.linalg.vector_norm(a)
    b_n = torch.linalg.vector_norm(b)
    if a_n == 0 or b_n == 0:
        return float("NaN")
    return float((a @ b) / (a_n * b_n))

def cos_similarity(T1: torch.Tensor, T2: torch.Tensor, i1: int, i2: int) -> None:
    if not (0 <= i1 < T1.shape[0]) or not (0 <= i2 < T2.shape[0]):
        raise IndexError(f"Indices out of range\n   T1={T1.shape[0]}, i1={i1}\n   T2={T2.shape[0]}, i2={i2}")
    sim = _cos(T1[i1], T2[i2])
    print(f"cos(T1[{i1}], T2[{i2}])={sim:.6}")

In [ ]:
T1 = _load_pt_file(Path("../smoketest/artifacts/pts/A8D0M1_ADE02.pt"))
T2 = _load_pt_file(Path("../smoketest/artifacts/pts/J9Z4E7_9ADEN.pt"))
T1_rows = T1.shape[0]
T2_rows = T2.shape[0]
limit = T1_rows if T1_rows < T2_rows else T2_rows
for i in range(limit):
    for j in range(limit):
        cos_similarity(T1, T2, i, j)

In [ ]:
long_seq = _load_pt_file(Path("../tests/smoketest/artifacts/pts/A0A077B1I6_CHIKV.pt"))
long_seq[2473]

In [ ]:
bp3_long_seq = torch.load(Path("../../data/7lj4_B.pt"))
bp3_long_seq = bp3_long_seq["representations"][33]
bp3_long_seq.shape